In [1]:
import warnings
from loguru import logger

warnings.simplefilter(action='ignore', category=FutureWarning)

from edgedroid.models.timings import *
import edgedroid.data as default_data

logger.enable("edgedroid")

data = default_data.load_curve_fitting_data()
data

,participant,prev_ttf,exec_time,prev_duration,neuro,neuroticism
0,134146,0.0,3.029,"[-inf, 5.0)","[-inf, 0.5)",0.375
1,134146,0.0,4.004,"[-inf, 5.0)","[-inf, 0.5)",0.375
2,134146,0.0,2.625,"[-inf, 5.0)","[-inf, 0.5)",0.375
3,134146,0.0,4.680,"[-inf, 5.0)","[-inf, 0.5)",0.375
4,134146,0.0,5.197,"[-inf, 5.0)","[-inf, 0.5)",0.375
...,...,...,...,...,...,...
6755,137353,0.0,6.358,"[5.0, 9.0)","[0.5, inf)",0.625
6756,137353,0.0,3.840,"[5.0, 9.0)","[0.5, inf)",0.625
6757,137353,0.0,2.932,"[9.0, inf)","[0.5, inf)",0.625
6758,137353,0.0,1.874,"[9.0, inf)","[0.5, inf)",0.625


In [2]:
from collections import deque
import pandas as pd
from typing import Tuple, Deque
from tqdm.notebook import tqdm
import numpy as np

result_rows = deque()
for participant in tqdm(data["participant"].unique(), desc="Run"):
    class MultiCurveFittingTestModel(MultiCurveFittingExecutionTimeModel):
        @staticmethod
        def get_data() -> pd.DataFrame:
            data = MultiCurveFittingExecutionTimeModel.get_data()
            return data[data["participant"] != participant].copy()
        
    class PowerFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (PowerFit,)
        
    class SquareFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (SquareFit,)
        
    class CubeFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (CubeFit,)
        
    class ExponentialFitTestModel(MultiCurveFittingTestModel):
        _fit_functions = (ExponentialFit,)
        
    def get_test_data(*args, **kwargs) -> (
        Tuple[
            pd.DataFrame,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
            pd.arrays.IntervalArray,
        ]
    ):
        data, *rest = ExecutionTimeModel.get_data()
        return (data[data["run_id"] != participant].copy(), *rest)

    class EmpiricalTestModel(EmpiricalETM):
        get_data = get_test_data
        
    class EmpiricalMeanTestModel(EmpiricalAggregateETM):
        get_data = get_test_data
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, aggregate_fn=np.mean, **kwargs)
            
    class EmpiricalMedianTestModel(EmpiricalAggregateETM):
        get_data = get_test_data
        
        def __init__(self, *args, **kwargs):
            super().__init__(*args, aggregate_fn=np.median, **kwargs)
        
    class TheoreticalTestModel(FittedETM):
        get_data = get_test_data

    class NaiveTestModel(FirstOrderETM):
        get_data = get_test_data
        
    class NaiveAggTestModel(FirstOrderAggregateETM):
        get_data = get_test_data
        
    cleanup = CleanupMode.TRUNCATE
    
    run_data = data[data["participant"] == participant].copy()

    neuroticism = run_data["neuroticism"].unique()[0]
    naive_model = NaiveTestModel()
    naive_mean_model = NaiveAggTestModel(np.mean)
    naive_median_model = NaiveAggTestModel(np.median)
    # curve_fitting_model = CurveFittingTestModel(neuroticism)
    
    power_fit_model = PowerFitTestModel(neuroticism)
    square_fit_model = SquareFitTestModel(neuroticism)
    cube_fit_model = CubeFitTestModel(neuroticism)
    exponential_fit_model = ExponentialFitTestModel(neuroticism)
    multi_fit_model = MultiCurveFittingExecutionTimeModel(neuroticism)
    
    # deque containing (model name, model object, ttf_bins, window_size, kernel name)
    models: Deque[Tuple[str, ExecutionTimeModel, int, int, str]] = deque()
    models.extend((
        ("1st order", naive_model, 0, 0, "none"),
        ("1st order (median)", naive_median_model, 0, 0, "none"),
        ("1st order (mean)", naive_mean_model, 0, 0, "none"),
        ("a * x^b + c", power_fit_model , 0, 0, "none"),
        ("a * x^2 + b * x + c", square_fit_model, 0, 0, "none"),
        ("a * x^3 + b * x^2 + c * x + d", cube_fit_model, 0, 0, "none"),
        ("a * e^x + b", exponential_fit_model, 0, 0, "none"),
        ("multi curve", multi_fit_model, 0, 0, "none")
    ))
    
    # for (ttf_bins, window_size) in itertools.product(range(1, 10), (4, 8, 12)):
    #     exp_kernel = ExponentialTTFWindowKernel(window_size=window_size)
    #     steep_linear_kernel = LinearTTFWindowKernel(window_size=window_size, max_relative_weight=10)
    #     shallow_linear_kernel = LinearTTFWindowKernel(window_size=window_size, max_relative_weight=2)
    #     average_kernel = AverageTTFWindowKernel(window_size=window_size)
    #     
    #     kernels = (
    #         ("exponential", exp_kernel),
    #         ("linear_steep", steep_linear_kernel),
    #         ("linear_shallow", shallow_linear_kernel),
    #         ("average", average_kernel)
    #     )
    #     model_classes = (
    #         ("empirical", EmpiricalTestModel),
    #         ("empirical (mean)", EmpiricalMeanTestModel),
    #         ("empirical (median)", EmpiricalMedianTestModel),
    #         ("theoretical", TheoreticalTestModel)
    #     )
    #     
    #     for (class_name, model_cls), (kernel_name, kernel) in itertools.product(model_classes, kernels):
    #         m = model_cls(kernel=kernel, neuroticism=neuroticism, ttf_levels=ttf_bins, cleanup=cleanup)
    #         models.append((class_name, m, ttf_bins, window_size, kernel_name))
        
    for _ in range(1):
        for i, (prev_ttf, exec_time, neuro, prev_duration) in enumerate(run_data[["prev_ttf", "exec_time", "neuro", "prev_duration"]].itertuples(index=False)):
            for model_name, model, ttf_bins, window_size, kernel_name in models:
                if i == 0:
                    # reset the model between runs
                    model.reset()
                else:
                    # first row has no previous ttf
                    model.advance(prev_ttf)
                    
                prediction = model.get_execution_time()
                error = prediction - exec_time
                result_rows.append({
                    "real": exec_time,
                    "prediction": prediction,
                    "error": error,
                    "sqr_error": np.square(error),
                    "model": model_name,
                    "duration": prev_duration,
                    "neuro": neuro,
                    # "kernel": kernel_name,
                    # "ttf_bins": ttf_bins,
                    # "window_size": window_size,
                })

results = pd.DataFrame(result_rows)
for col in ("model", ):  # "kernel"):
    results[col] = results[col].astype("category")

results.to_parquet("./full_validation.parquet")
results

Run:   0%|          | 0/40 [00:00<?, ?it/s]

2024-01-09 13:02:14.019 | DEBUG    | edgedroid.models.timings.curve:__init__:114 - Curve fitting aggregation function: mean
2024-01-09 13:02:14.079 | INFO     | edgedroid.models.timings.curve:__init__:139 - Fitting execution time functions to data...
2024-01-09 13:02:14.108 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [-inf, 5.0): a * x^b + c (MSE: 0.03, prev. MSE 0.03)
2024-01-09 13:02:14.132 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [5.0, 9.0): a * x^b + c (MSE: 0.08, prev. MSE 0.08)
2024-01-09 13:02:14.136 | INFO     | edgedroid.models.timings.curve:__init__:147 - New best fit function for duration [9.0, inf): a * x^b + c (MSE: 0.07, prev. MSE 0.07)
2024-01-09 13:02:14.137 | DEBUG    | edgedroid.models.timings.curve:__init__:114 - Curve fitting aggregation function: mean
2024-01-09 13:02:14.146 | INFO     | edgedroid.models.timings.curve:__init__:139 - Fitting execution time function

,real,prediction,error,sqr_error,model,duration,neuro
0,3.029,3.140389,0.111389,0.012407,1st order,"[-inf, 5.0)","[-inf, 0.5)"
1,3.029,5.261648,2.232648,4.984719,1st order (median),"[-inf, 5.0)","[-inf, 0.5)"
2,3.029,5.873496,2.844496,8.091159,1st order (mean),"[-inf, 5.0)","[-inf, 0.5)"
3,3.029,5.477779,2.448779,5.996520,a * x^b + c,"[-inf, 5.0)","[-inf, 0.5)"
4,3.029,5.247540,2.218540,4.921919,a * x^2 + b * x + c,"[-inf, 5.0)","[-inf, 0.5)"
...,...,...,...,...,...,...,...
54075,4.241,3.666877,-0.574123,0.329617,a * x^b + c,"[9.0, inf)","[0.5, inf)"
54076,4.241,3.651962,-0.589038,0.346965,a * x^2 + b * x + c,"[9.0, inf)","[0.5, inf)"
54077,4.241,3.236597,-1.004403,1.008825,a * x^3 + b * x^2 + c * x + d,"[9.0, inf)","[0.5, inf)"
54078,4.241,4.127838,-0.113162,0.012806,a * e^x + b,"[9.0, inf)","[0.5, inf)"


In [3]:
# results.groupby(["model", "kernel", "window_size", "ttf_bins"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

results.groupby(["model", "neuro", "duration"], observed=True)["sqr_error"].mean().sort_values(ascending=True)

# look at neuroticism
# look at durations
# look different functions
# look at delta in ttf to reset duration (maybe not instantaneous delta)

# look at different curves for each quadrant

model                          neuro        duration   
multi curve                    [-inf, 0.5)  [5.0, 9.0)      6.709440
a * x^b + c                    [-inf, 0.5)  [5.0, 9.0)      6.795480
a * e^x + b                    [-inf, 0.5)  [5.0, 9.0)      6.797602
a * x^3 + b * x^2 + c * x + d  [-inf, 0.5)  [5.0, 9.0)      6.799207
a * x^2 + b * x + c            [-inf, 0.5)  [5.0, 9.0)      6.827645
1st order (median)             [-inf, 0.5)  [5.0, 9.0)      7.006482
1st order (mean)               [-inf, 0.5)  [5.0, 9.0)      7.876696
multi curve                    [0.5, inf)   [5.0, 9.0)      8.707317
a * x^b + c                    [0.5, inf)   [5.0, 9.0)      8.794988
a * x^3 + b * x^2 + c * x + d  [0.5, inf)   [5.0, 9.0)      8.812825
a * e^x + b                    [0.5, inf)   [5.0, 9.0)      8.816071
a * x^2 + b * x + c            [0.5, inf)   [5.0, 9.0)      8.819612
1st order (median)             [0.5, inf)   [5.0, 9.0)      8.980667
multi curve                    [-inf, 0.5)  [-i

In [4]:
results.groupby(["model"], observed=True)["sqr_error"].describe().sort_values(by="mean", ascending=True)


,count,mean,std,min,25%,50%,75%,max
model,,,,,,,,
multi curve,6760.0,9.549134,55.029762,1.863419e-09,0.532560,2.463335,6.890643,2309.083742
a * x^3 + b * x^2 + c * x + d,6760.0,9.682107,55.653272,2.118783e-07,0.545524,2.490341,6.940064,2337.240573
a * e^x + b,6760.0,9.695366,55.487454,4.387432e-07,0.564402,2.470012,7.019435,2328.702358
a * x^2 + b * x + c,6760.0,9.709345,55.805685,1.742164e-07,0.577859,2.429675,6.817916,2348.227132
a * x^b + c,6760.0,9.722399,56.092836,5.012343e-07,0.588717,2.476210,6.867841,2344.899237
1st order (median),6760.0,10.084425,58.281182,6.192399e-08,0.542842,2.755124,6.592557,2507.646370
1st order (mean),6760.0,10.268799,55.853281,1.201064e-07,0.982974,3.482772,8.450134,2448.670017
1st order,6760.0,19.446531,78.359706,2.617509e-08,1.001839,4.576765,15.059831,2663.781908


In [5]:
# # %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns
# 
# fg = sns.catplot(results, kind="point", x="ttf_bins", y="sqr_error", hue="model_tag", height=5, aspect=2)
# fg.set(ylim=(0, None))
# fg.set_ylabels("MSE")
# for ax in fg.axes.flat:
#     ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=45, ha='right')
# plt.show()

In [6]:
# results.groupby(["model", "ttf_bins", "cleanup"])["sqr_error"].mean().sort_values()